In [10]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.proxy import Proxy, ProxyType
import time

#proxy configuration for not getting 403 status
prox = Proxy()
prox.proxy_type = ProxyType.MANUAL


#driver for dynamically using chrome for scraping
capabilities = webdriver.DesiredCapabilities.CHROME
prox.add_to_capabilities(capabilities)

driver = webdriver.Chrome(desired_capabilities=capabilities)

def get_key(my_dict,val): 
    for key, value in my_dict.items(): 
         if val == value: 
            return key 
    return "key doesn't exist"
        
def getBuilding(link):

    driver.get(link)
    driver.implicitly_wait(8)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #find search results
    main_div = soup.find_all('div', {'id':'projects'})
    
    lib_name_desc = dict()
    DESCS=[]
    detail_links = dict()
    for div in main_div :
        library_titles = div.find_all('div', {'class':'title'})

            

        for elem in library_titles:    
            details = elem.find_all('a')
            for info in  details:
                if 'details' in info['href']:  
                    detail_links[info.string] = info['href']

                    
           
                    

    lib_name_link = dict()
    for dl in detail_links.values():
        lib_name = get_key(detail_links, dl)
        link = 'https://android-arsenal.com' + dl
        driver.get(link)
        driver.implicitly_wait(3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        library_details = soup.find_all('dl' ,{'class':'dl-horizontal'})
        
        for ld in library_details:
            library_website_component = ld.find_all('a', {'rel': 'nofollow', 'target': '_blank'})
            
            for lwd in library_website_component:
                
                if 'http' in str(lwd.string):
                    lib_name_link[lib_name] = lwd.string
                    #print(lib_name, ':', lwd.string)

                    
        df1 = pd.DataFrame(list(zip( list(lib_name_link.keys()), list(lib_name_link.values()))), columns =['Name', 'Link']) 
        df1.to_csv('/Users/beyzacevik/Downloads/arsenal-library-name-database1.csv')
    
getBuilding('https://android-arsenal.com/tag/25?sort=rating')


